# Twitter Crawler: notebook to search institutional twitter accounts of mexican states

**input** : state names \
**output code**:
    - Data Frame with: 
        search parameter
        twitter account found 
        profile description

In [45]:
def make_query_twitter(string):
    """
    Function to make query search for twitter
    
    Parameters
    ----------
    string: str
        string with name of Mexico State
        
    Returns
    -------
    query: str
        
    """
    try:
        name = re.sub(' ', '%20', string)
        query = 'https://twitter.com/search?q=' + name +'&src=typed_query&f=user'
        return(query)
    except:
        return('filter')

In [19]:
from selenium.webdriver import Chrome
import numpy as np
from uuid import uuid4
import re
import itertools
import functools
import operator
from datetime import date, timedelta
import pandas as pd
import json
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from twitter_scraper import Profile
import pandas as pd
from datetime import date, timedelta
import os
from tqdm import tqdm
from os import path
import time

In [67]:
def get_twitter_account(x):
    try:
        driver.get(x)
        driver.implicitly_wait(4)
        path = driver.find_element_by_css_selector('#react-root > div > div > div.css-1dbjc4n.r-13qz1uu.r-417010 > main > div > div > div > div > div > div > div > div > section > div > div > div:nth-child(1) > div > div > div > div.css-1dbjc4n.r-1iusvr4.r-16y2uox.r-5f2r5o > div > div.css-1dbjc4n.r-1wbh5a2.r-dnmrzs > a').text
        username = re.findall('@(.*)', path)[0]
        return(username)
    except:
        return('no account found')
    

## Input Data

In [3]:
estados = pd.read_csv('./data/raw/entidades.csv')

In [9]:
estados = estados[['entidad_name']].drop_duplicates()

In [68]:
estados['entidad_name'] = estados.apply(lambda x: x['entidad_name'].lower(), axis = 1)
estados['input_string'] = estados.apply(lambda x: 'gobierno ' + x['entidad_name'].lower() , axis = 1)
estados['input_string_salud'] = estados.apply(lambda x: 'secretaria salud ' + x['entidad_name'].lower() , axis = 1)
estados['query'] = estados.apply(lambda x: make_query_twitter(x['input_string']), axis = 1)
estados['query_salud'] = estados.apply(lambda x: make_query_twitter(x['input_string_salud']), axis = 1)

In [59]:
webdriver = "/usr/local/bin/chromedriver"
driver = Chrome(webdriver)

In [66]:
list_estados = []
with tqdm(total = len(list(estados['query']))) as pbar:
    for x, y in zip(list(estados['query']), list(estados['entidad_name'])):
        pbar.update(1)
        
        username = get_twitter_account(x)
        user = re.sub('https://twitter.com/', '', username)
        temp = {}
        
        if username != 'no account found':
            profile = Profile(user).to_dict()
            temp['entidad_name'] = y
            temp['username_twitter'] = username
            temp['name_twitter'] = profile['name']
            temp['biography'] = profile['biography']
            temp['website'] = profile['website']
            temp['last_update'] = date.today().strftime("%Y-%m-%d")
        else:
            temp['entidad_name'] = y
            temp['username_twitter'] = username
            temp['name_twitter'] = 'no account found'
            temp['biography'] = 'no account found'
            temp['website'] = 'no account found'
            temp['last_update'] = date.today().strftime("%Y-%m-%d")
            
        list_estados.append(temp)

100%|██████████| 32/32 [04:49<00:00,  9.04s/it]


In [69]:
estados_twitters = pd.DataFrame(list_estados)

In [71]:
list_estados_salud = []
with tqdm(total = len(list(estados['query_salud']))) as pbar:
    for x, y in zip(list(estados['query_salud']), list(estados['entidad_name'])):
        pbar.update(1)
        
        username = get_twitter_account(x)
        user = re.sub('https://twitter.com/', '', username)
        temp = {}
        
        if username != 'no account found':
            profile = Profile(user).to_dict()
            temp['entidad_name'] = y
            temp['username_twitter'] = username
            temp['name_twitter'] = profile['name']
            temp['biography'] = profile['biography']
            temp['website'] = profile['website']
            temp['last_update'] = date.today().strftime("%Y-%m-%d")
        else:
            temp['entidad_name'] = y
            temp['username_twitter'] = username
            temp['name_twitter'] = 'no account found'
            temp['biography'] = 'no account found'
            temp['website'] = 'no account found'
            temp['last_update'] = date.today().strftime("%Y-%m-%d")
            
        list_estados_salud.append(temp)

100%|██████████| 32/32 [05:54<00:00, 11.07s/it]


In [74]:
estados_twitters_salud = pd.DataFrame(list_estados_salud).rename(columns = {'username_twitter':'twitter_salud', 
                                                                        'name_twitter':'name_twitter_salud',
                                                                        'biography':'biography_salud',
                                                                        'website':'website_salud'}).drop(['last_update'], axis = 1)

In [72]:
## 

In [77]:
twitter_accounts = pd.merge(left = estados_twitters,
        right = estados_twitters_salud,
        on = 'entidad_name', 
        how = 'inner')

In [78]:
twitter_accounts.to_csv('./data/raw/twitter_accounts.csv', encoding = 'utf-8 sig')